# 미니 의도분석기 실습

## 학습목차

0. 필요 라이브러리 임포트
 - python, konlpy, pandas, Komoran
1. 사전 데이터 확인
 - 개체명 파일, 액션-개체명 파일, 액션-패턴 파일
2. 발화 형태소 분석 (Komoran 형태소 분석기 사용)
 - 액션-패턴 파일 변경하여 저장
3. 개체명 검색
 - 일치하는 개체명이 있을 시 치환 (ex. 에어컨 -> iot_device)
4. 패턴 검색
 - 일치하는 패턴 발견 시 액션 반환, 발견하지 못하면 None 반환
5. 새로운 액션, 패턴 추가

# 0. 필요 라이브러리 임포트

### python, konlpy

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install konlpy

### pandas, Komoran

In [2]:
import pandas as pd
from konlpy.tag import Komoran

# 1. 사전 데이터 확인

### <font color=green> 구글 드라이브 연동 </font>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd "/content/drive/My Drive/lecture"
!git clone "https://github.com/friendly036/lecture_test.git"

/content/drive/My Drive/lecture
Cloning into 'lecture_test'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 8 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), 6.90 KiB | 57.00 KiB/s, done.


In [7]:
%cd "/content/drive/My Drive/lecture/lecture_test"
%ls

/content/drive/My Drive/lecture/lecture_test
action_ne.csv  action_pattern.csv  mini_nlp_Q.ipynb  ne_value.csv


### <font color=green>개체명 파일 : ne_value.csv</font>

In [8]:
ne_value = pd.read_csv('ne_value.csv', encoding='utf-8')

In [9]:
ne_value

,ne,value
0,iot_device,에어컨
1,iot_device,공기청정기
2,dt_day,오늘
3,dt_day,내일
4,contents_title,런닝맨
5,contents_title,아바타
6,contents_title,기생충


### <font color=green>액션-개체명 파일 : action_ne.csv</font>

In [10]:
action_ne = pd.read_csv('action_ne.csv', encoding='utf-8')

In [11]:
action_ne

,action,ne
0,volume_up,NaN
1,volume_down,NaN
2,iot_device_power_on,iot_device
3,iot_device_power_off,iot_device
4,search_weather,dt_day
5,search_contents,contents_title
6,search_director,contents_title


### <font color=green> 액션-패턴 파일 : action_pattern.csv </font>

In [26]:
action_pattern = pd.read_csv('action_pattern.csv', encoding='utf-8')

In [27]:
action_pattern

,action,pattern
0,volume_up,볼륨 올려
1,volume_down,볼륨 내려
2,iot_device_power_on,iot_device 켜줘
3,iot_device_power_off,iot_device 꺼줘
4,search_weather,dt_day 날씨 알려줘
5,search_contents,contents_title 검색해줘
6,search_director,contents_title 감독 누구야


# 2. 발화 형태소 분석

### <font color=green>Komoran tag 종류 확인</font>

In [ ]:
komoran = Komoran()
komoran.tagset

### <font color=green> Komoran 사용 예시</font>

In [ ]:
komoran.pos("KOMORAN은 한국어 형태소 분석기입니다")

### <font color=green> 액션-패턴 파일 변경하여 저장</font>

##### <font color=blue> **[문제 1] 액션-패턴 파일의 내용을 komoran pos 함수를 이용하여 형태소 분석된 내용으로 "pattern" 컬럼에 저장하세요.** </font>

In [28]:
# 읽어 들일 파일명 : action_pattern.csv
# 액션-패턴 정보(action_pattern) 중 "pattern" 내용에 대해 형태소 분석 내용 반영
# 결과 : "action_pattern_komoran.csv" 파일로 저장

# 변경 내용 (volume_up 등의 action은 본형태 유지)
## volume_up  볼륨 올려 -> volume_up  볼륨 올리 어
## search_weather	dt_day 날씨 알려줘 -> search_weather	dt_day 날씨 알리

action_pattern_komoran = action_pattern

def create_pos_pattern(text):
  text = komoran.pos(text)
  words = []
  patterns = []
  
  for word in text:
    if(word[1] in ["SL", "SW"]): # dt_day 등의 NE 명칭은 형태 유지
      words.append(word[0])
    elif(word[1] not in ["EC", "VX"]): # TODO : komoran pos 함수 활용 - '연결 어미'(EC), '보조 용언'(VX) 제거
      words.append(" " + word[0])

  pattern = ''.join(words).strip()
  return pattern

action_pattern_komoran["pattern"] = action_pattern_komoran["pattern"].apply(lambda x: create_pos_pattern(x))
action_pattern_komoran.to_csv('action_pattern_komoran.csv', index=False)

In [29]:
action_pattern_komoran = pd.read_csv('action_pattern_komoran.csv', encoding='utf-8')
action_pattern_komoran

## 기대 결과
# action	pattern
# 0	tv_volume_up	볼륨 올 리
# 1	tv_volume_down	볼륨 내리
# 2	sh_device_power_on	iot_device 켜

,action,pattern
0,volume_up,볼륨 올리
1,volume_down,볼륨 내리
2,iot_device_power_on,iot_device 켜
3,iot_device_power_off,iot_device 끄
4,search_weather,dt_day 날씨 알리
5,search_contents,contents_title 검색 하
6,search_director,contents_title 감독 누구 야


# 3. 개체명 검색


##### <font color=blue> **[문제 2] 분석된 형태소 중 명사에 한하여 개체명을 검색하여 치환하세요.** </font>

In [35]:
# 읽어 들일 파일명 : ne_value.csv
# 함수명 : replace_ne()
# 개체명이 치환된 패턴을 반환

ne_value = pd.read_csv('ne_value.csv', encoding='utf-8')

def replace_ne(text):
  pos = komoran.pos(text)

  for word in pos:
    if(word[1] in ["NNP","NNG"]): # TODO : komoran pos 함수 활용, "명사"에 한하여 개체명을 검색한 후 개체명 이름(ne)로 치환
      for row in ne_value.itertuples():
          if(word[0] == row.value):
            text = text.replace(row.value, row.ne)
  return text

text = replace_ne("내일 날씨 알려줘")
text

## 기대 결과
# 'dt_day 날씨 알려줘'

'dt_day 날씨 알려줘'

# 4. 패턴 검색

##### <font color=blue> **[문제 3] 입력되는 발화에 일치되는 패턴을 있는지 검색하여 결과를 출력하세요.** </font>

In [41]:
# 읽어 들일 파일명 : action_pattern_komoran.csv, action_ne.csv
# 함수명 : detect_pattern()

# action_ne.csv 파일에 선언된 action을 모두 점검
# 해당 action 에서 허용하는 NE인지 확인 후 개체명 치환 (action_ne.csv)
# 일치하는 패턴 발견 시 액션 반환, 발견하지 못하면 None 반환

action_ne = pd.read_csv('action_ne.csv', encoding='utf-8')

patterns = pd.read_csv('action_pattern_komoran.csv', encoding='utf-8')
pattern_action = pd.Series(patterns.action.values, index=patterns.pattern).to_dict()

def detect_pattern(text):
  result_dict = {"pos":'', "matched pattern":'None', "detected action":'None'}
  pos = komoran.pos(text)
  pos_text = create_pos_pattern(text)
  result_dict["pos"] = pos

  for actions in action_ne.itertuples():
    
    # 해당 action 에서 허용하는 NE인지 확인 후 개체명 치환
    for word in pos:
      if(word[1] in ["NNP","NNG"]): # TODO : [문제 2]번 결과
        for row in ne_value.itertuples():
          if(word[0] == row.value and not pd.isna(actions.ne) and row.ne == actions.ne):
            pos_text = pos_text.replace(row.value, row.ne)

    # 일치하는 패턴이 있는지 확인
    for pattern, action in pattern_action.items():
      # TODO : 개채명이 치환된 패턴과 action_pattern_komoran 파일 내 패턴을 비교하여 같은 경우, 해당 패턴과 action을 result_dict 결과에 저장
      if pos_text == pattern:
        result_dict["matched pattern"] = pattern
        result_dict["detected action"] = action

  return result_dict

test_pattern = "오늘 날씨 알려줘"
result = detect_pattern(test_pattern)

print("test pattern: ", test_pattern)
print("pos: ", result["pos"])
print("matched pattern: ", result["matched pattern"])
print("detected action: ", result["detected action"])


# 기대 결과
# test pattern:  오늘 날씨 알려줘
# pos:  [('오늘', 'NNG'), ('날씨', 'NNP'), ('알리', 'VV'), ('어', 'EC'), ('주', 'VX'), ('어', 'EC')]
# matched pattern:  dt_day 날씨 알리
# detected action:  wt_search_weather

test pattern:  오늘 날씨 알려줘
pos:  [('오늘', 'NNG'), ('날씨', 'NNP'), ('알리', 'VV'), ('어', 'EC'), ('주', 'VX'), ('어', 'EC')]
matched pattern:  dt_day 날씨 알리
detected action:  search_weather


# 5. 새로운 액션 패턴 추가

##### <font color=blue> **[문제 4] "기생충 감독 누구야" 발화 인식하기 위한 액션, 개체명, 패턴을 추가하세요.** </font>

In [40]:
# 액션 : "search_director"
# 발화 : "기생충 감독 누구야"

# TODO : 파일 수정 - ne_value.csv. action_ne.csv, action_pattern.csv
# 파일 수정 후 2~4번 과정 통해 의도분석 결과 확인

test_pattern = "기생충 감독 누구야"
result = detect_pattern(test_pattern)

print("test pattern: ", test_pattern)
print("pos: ", result["pos"])
print("matched pattern: ", result["matched pattern"])
print("detected action: ", result["detected action"])

test pattern:  기생충 감독 누구야
pos:  [('기생충', 'NNG'), ('감독', 'NNG'), ('누구', 'NP'), ('야', 'JX')]
matched pattern:  contents_title 감독 누구 야
detected action:  search_director
